In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import stop_words
import os
from nltk.corpus import stopwords 
import nltk
import re
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
#nltk.download('stopwords')

In [8]:
stop = set(['?',',',':','@']) 

In [9]:
ROOT = '../data/'

QA_dict = {}

In [10]:
os.listdir(ROOT)

['CDC.xlsx',
 'Coronavirus (COVID-19) frequently asked questions _ CDC.pdf',
 'covidquestionsca.xlsx',
 'EAC.xlsx',
 'JHU.xlsx',
 'MOHKE.xlsx',
 'sources.txt',
 'WHO.xlsx']

In [11]:
paths = [ROOT+d for d in os.listdir(ROOT) if d.endswith(".xlsx")]

In [12]:
paths

['../data/CDC.xlsx',
 '../data/covidquestionsca.xlsx',
 '../data/EAC.xlsx',
 '../data/JHU.xlsx',
 '../data/MOHKE.xlsx',
 '../data/WHO.xlsx']

In [13]:
data = [ pd.read_excel(p, names = ['q','a']) for p in paths]

In [14]:
QA = pd.DataFrame()

In [15]:
for d in data[:]:
    if d.shape[0]>0:
        print(d.shape)
        QA = pd.concat((QA,d),axis = 0)

(116, 2)
(10, 2)
(26, 2)
(86, 2)
(85, 2)


In [16]:
QA.shape

(323, 2)

In [17]:
QA.sample(10)

,q,a
67,What are the symptoms and complications that C...,People with COVID-19 have had a wide range of ...
21,Am I at risk for COVID-19 in the United States?,This is a rapidly evolving situation and the r...
7,Why do the number of cases for previous days i...,Delays in reporting can cause the number of CO...
85,Are residents of homeless shelters a vulnerabl...,Yes. Homeless shelters are often overcrowded a...
33,Should children wear masks?\n,CDC recommends that everyone 2 years and older...
56,When to Seek Emergency Medical Attention\n,Look for emergency warning signs* for COVID-19...
78,How are COVID-19 and influenza viruses different?,Mortality for COVID-19 appears higher than for...
86,What do funeral home workers need to know abou...,A funeral or visitation service can be held fo...
50,What if my child or someone else in the home i...,If your child with special healthcare needs be...
76,How are COVID-19 and influenza viruses different?,The speed of transmission is an important poin...


In [18]:
 snow = nltk.stem.SnowballStemmer('english')

In [19]:
def clean(q_a):
    temp = []
    snow = nltk.stem.SnowballStemmer('english')
    for sentence in q_a:
        sentence = str(sentence)
        sentence = sentence.lower()                 # Converting to lowercase
        cleanr = re.compile('<.*?>')
        sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
        sentence = re.sub(r'[?|!|\'|"|#]\d',r'',sentence)
        sentence = re.sub(r'[.|,|)|(|\|/]@',r' ',sentence)        #Removing Punctuations
        words = [snow.stem(word) for word in sentence.split() if word not in stop]   # Stemming and removing stopwords
        temp.append(words)
    q_a = temp  
    return q_a

In [20]:
def clean_col(c):
    sent = []
    for row in c:
        sequ = ''
        for word in row:
            sequ = sequ + ' ' + word
        sent.append(sequ)
    c = sent
    return c

In [21]:
q_new =  clean_col(clean(QA['q']))
a_new =  clean_col(clean(QA['a']))

In [22]:
QA['q_new'] =  q_new
QA['a_new'] = a_new

In [23]:
QA.head()

,q,a,q_new,a_new
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...,what is a novel coronavirus?,a novel coronavirus is a new coronavirus that...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza...",whi is the diseas be call coronavirus diseas ...,"on februari 11, 2020 the world health organ a..."
2,Why might someone blame or avoid individuals a...,People in the U.S. may be worried or anxious a...,whi might someon blame or avoid individu and ...,peopl in the u.s. may be worri or anxious abo...
3,How can people help stop stigma related to COV...,People can ght stigma by providing social supp...,how can peopl help stop stigma relat to covid...,peopl can ght stigma by provid social support...
4,Why do some state’s COVID-19 case numbers some...,CDC’s overall case numbers are validated throu...,whi do some state covid-19 case number someti...,cdc overal case number are valid through a co...


In [27]:
count_vect = CountVectorizer()
bow_data = count_vect.fit_transform(QA['q_new'])

In [29]:
tq = ["cornonavirus"]
tq = clean(tq)
tq = clean_col(tq)

In [30]:
r = count_vect.transform(tq)

In [28]:
for i,(q , a) in enumerate(zip(QA['q'],QA['a'])):
    QA_dict[i] = [q,a]                   

In [37]:
def cv_test(q):
    tq = [q]
    tq = clean(tq)
    tq = clean_col(tq)
    r = count_vect.transform(tq)
    return r

In [38]:
QA['q'].sample(20)

21    Protection measures for persons who are in or ...
94    Who should clean and disinfect community space...
29    Should contact lens wearers take special preca...
87    What should I do if my family member died from...
25    What makes this upper respiratory infection ca...
59    How should I help a friend, neighbor, or famil...
38    What is the difference between molecular tests...
30    How is COVID-19 is impacting people without sp...
4                    What are the symptoms of COVID-19?
59    Should I use soap and water or a hand sanitize...
15                 Is there a vaccine against COVID-19?
20    Is there a difference between hospital-acquire...
19                     Can I attend the church service?
71    Do pregnant women with suspected or confirmed ...
93    Is cleaning alone e ective against the virus t...
71    Many of these viruses come from bats, but bats...
47    Can humans be infected with the COVID-19 from ...
12                 Why are we seeing a rise in c

In [39]:
QA.shape[0] == bow_data.shape[0]

True

In [41]:
bow_data[0].shape

(1, 723)

In [43]:
r.shape

(1, 723)

In [ ]:
while True:
    q = input("Q: ")
    q = cv_test(q)
    ranks = np.array([cosine_similarity(q,bow_data[i]) for i in range(QA.shape[0])]).reshape(QA.shape[0],1)
    loc = np.argmax(ranks)  
    a = QA_dict[loc][1]
    print("A: ",a)

Q: what is coronavirus
A:  Coronaviruses are a large family of viruses which occur naturally in animals and may cause illness in animals and humans. In humans, coronaviruses cause respiratory infections ranging from common colds with mild symptoms to more severe diseases such as the Middle East Respiratory Syndrome (MERS) and the Severe Acute Respiratory Syndrome (SARS). The most recently discovered virus in this family causes the Coronavirus Disease, COVID-19.
Q: what is covi-19
A:  The Coronavirus Disease is a flu-like respiratory disease that is highly contagious.
Q: what is covid-19
A:  The Coronavirus Disease is a flu-like respiratory disease that is highly contagious.
Q: keep my family safe
A:  Watch your child for any signs of illness. If you see any sign of illness consistent with symptoms of COVID-19, particularly fever, cough, or shortness of breath, call your healthcare provider and keep your child at home and away from others as much as possible.Follow CDC’s guidance on “Wh